## Fig. 2 panel D umap
This notebook calculates 2D and 3D UMAP embeddings and generate plots

In [1]:
import os, sys
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import plotly.io as pio
import umap
from sklearn.decomposition import PCA

plt.style.use('ggplot')
plt.rcParams['pdf.fonttype'] = 42

script_path = Path.cwd().parent.parent.parent / "script"
data_path = Path.cwd().parent.parent.parent / "data"
sys.path.append(str(script_path))
from pyseus.plotting import plotly_umap as pu

save_path = Path.cwd() / "output"
if not os.path.exists(save_path):
    os.makedirs(save_path)

### load data

In [2]:
%store -r timestamp
print(f"Timestamp: {timestamp}") 

Timestamp: 2023-10-21-imp5-for-figures


In [3]:
# define files to load
input_dir = Path.cwd().parent / "panel_C" / "output"
umap_table_path = input_dir / f"{timestamp}_umap_table.csv"
quants_path = input_dir / f"{timestamp}_quants.csv"

# load data
try:
    # load the file
    umap_table = pd.read_csv(umap_table_path, index_col=0)
except FileNotFoundError:
    print(f"File {umap_table_path} not found.\nPlease run Fig2_C_consensus_annotation.ipynb first or specify the correct timestamp, current value is {timestamp}")
except pd.errors.ParserError:
    print(f"There was an error parsing the CSV file at {umap_table_path}.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

try:
    # load the file
    quants = pd.read_csv(quants_path, index_col=0)
except FileNotFoundError:
    print(f"File {quants_path} not found.\nPlease run Fig2_C_consensus_annotation.ipynb first or specify the correct timestamp, current value is {timestamp}")
except pd.errors.ParserError:
    print(f"There was an error parsing the CSV file at {quants_path}.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

### preprocessing

In [4]:
# scale the table for UMAP
scaled = pu.scale_table(matrix=quants, method='standard')

### PCA
for exploratory purposes only, and the principle components are not used in downstream analysis

In [5]:
# perform PCA
pca = PCA(n_components=10)
pca.fit(scaled)

umap_table['PC_1'] = pca.transform(scaled)[:, 0]
umap_table['PC_2'] = pca.transform(scaled)[:, 1]

In [6]:
# Variance explained by each principal component
variance_explained = pca.explained_variance_ratio_
variance_explained

array([0.22243451, 0.16366524, 0.06701723, 0.04919012, 0.03951465,
       0.03162415, 0.03144586, 0.02875264, 0.02148553, 0.02086402])

In [7]:
# generate 2D PCA plot
label_to_color = "consensus_graph_annnotation"  # **choose which annotation column to highlight here** , other choices: cluster_annotation, Protein-level_consensus_annotation

fig = pu.interaction_umap(
    umap_table,
    node_name="Gene_name_canonical", 
    cluster=label_to_color, opacity=0.35, unlabelled_color="#D0D3D4", unlabelled_opacity=0.1,
    pointsize=6, x="PC_1", y="PC_2",
    categorical=True,
)
fig.update_layout(width=1200, height=800)
# add title
fig.update_layout(title=f"PCA")

fig.show()

# save the figure as an html file
save_path = os.path.join("output")
save_name = f"{timestamp}_PCA_2Dview.html"
pio.write_html(fig, file=os.path.join(save_path, save_name), auto_open=False)

### UMAP

In [8]:
# UMAP parameters
n_neighbors = 20
min_dist = 0.1
metric = 'euclidean'

# flip the umap coordinates (if needed)
flip = True

# set the UMAP seed
UMAP_seed = 1234

In [9]:
# calculate 2D UMAP embeddings
fit = umap.UMAP(
    n_neighbors=n_neighbors, 
    min_dist=min_dist, 
    metric=metric, 
    random_state=UMAP_seed
)
u = fit.fit_transform(scaled)
umap_table['umap_1'] = u[:, 0] 
umap_table['umap_2'] = u[:, 1]

# flip the UMAP coordinates
if flip:
    max_x = max(umap_table["umap_1"])
    max_y = max(umap_table["umap_2"])
    umap_table["umap_1"] = max_x - umap_table["umap_1"]
    umap_table["umap_2"] = max_y - umap_table["umap_2"]


# calculate 3D UMAP embeddings
fit3D = umap.UMAP(
    n_neighbors=n_neighbors, 
    min_dist=min_dist, 
    metric=metric, 
    random_state=UMAP_seed, 
    n_components=3
)
u3D = fit3D.fit_transform(scaled)
# add the UMAP coordinates to the table
umap_table['3D_umap_1'] = u3D[:, 0] 
umap_table['3D_umap_2'] = u3D[:, 1]
umap_table['3D_umap_3'] = u3D[:, 2]

### save UMAP embeddings

In [10]:
# save umap embedding to csv file
save_name = f"{timestamp}_UMAP_embeddings_seed={UMAP_seed}.csv"
umap_table.to_csv(os.path.join(save_path, save_name), index=False)

In [11]:
# save scaled quants to csv file
save_name = f"{timestamp}_scaled_quants.csv"
umap_table.iloc[:, 8:8+scaled.shape[1]] = scaled
umap_table.to_csv(os.path.join(save_path, save_name), index=False)

# save 0-1 scaled quants to csv file
def normalize_rowwise(df):
    row_sums = df.sum(axis=1)
    normalized_df = df.div(row_sums, axis=0)
    return normalized_df
save_name = f"{timestamp}_01_scaled_quants.csv"
data = umap_table.iloc[:, 8:69].copy()
data[data<0] = 0
umap_table.iloc[:, 8:69] = normalize_rowwise(data)
umap_table.to_csv(os.path.join(save_path, save_name), index=False)

### generate UMAP plots

In [12]:
# plot the 2D UMAP
label_to_color = "consensus_graph_annnotation"  # **choose which annotation column to highlight here** , other choices: cluster_annotation, Protein-level_consensus_annotation

fig = pu.interaction_umap(
    umap_table,
    node_name="Gene_name_canonical", 
    cluster=label_to_color, opacity=0.35, unlabelled_color="#D0D3D4", unlabelled_opacity=0.1,
    pointsize=6, x="umap_1", y="umap_2",
    categorical=True,
)
fig.update_layout(width=1200, height=800)

fig.show()

# save the figure as an html file
save_path = os.path.join("output")
save_name = f"{timestamp}_UMAP_2Dview_seed={UMAP_seed}html"
pio.write_html(fig, file=os.path.join(save_path, save_name), auto_open=False)

In [13]:
# plot the 3D UMAP
label_to_color = "consensus_graph_annnotation"  # **choose which annotation column to highlight here**

fig = pu.interaction_3D_umap(
    umap_table,
    node_name="Gene_name_canonical",
    cluster=label_to_color, unlabelled_color="#D0D3D4", unlabelled_opacity=0.1,
    x="3D_umap_1", y="3D_umap_2", z="3D_umap_3",
    categorical=True,
)
fig.update_layout(width=1200, height=800)

fig.show()

# save the figure as an html file
save_path = os.path.join("output")
save_name = f"{timestamp}_UMAP_3Dview_seed={UMAP_seed}.html"
pio.write_html(fig, file=os.path.join(save_path, save_name), auto_open=False)